##### Copyright 2021 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Leitor do Apache ORC

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/io/tutorials/orc"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/orc.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/orc.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/io/tutorials/orc.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

O Apache ORC é um formato popular de armazenamento baseado em colunas. O pacote tensorflow-io conta com uma implementação padrão de leitura de arquivos do [Apache ORC](https://orc.apache.org/).

## Configuração

Instale os pacotes necessários e reinicie o runtime


In [2]:
!pip install tensorflow-io

In [3]:
import tensorflow as tf
import tensorflow_io as tfio

### Baixe um arquivo de dataset de amostra em ORC

O dataset que usaremos está no [Iris Dataset](https://archive.ics.uci.edu/ml/datasets/iris) da UCI e contém 3 classes com 50 instâncias cada, em que cada classe se refere a um tipo de planta íris. Ele tem 4 atributos: (1) sepal length (comprimento da sépala), (2) sepal width (largura da sépala), (3) petal length (comprimento da pétala) e (4) petal width (largura da pétala), e a última coluna contém o rótulo de classe.

In [4]:
!curl -OL https://github.com/tensorflow/io/raw/master/tests/test_orc/iris.orc
!ls -l iris.orc

## Crie um dataset a partir do arquivo

In [35]:
dataset = tfio.IODataset.from_orc("iris.orc", capacity=15).batch(1)

Avalie o dataset:

In [42]:
for item in dataset.take(1):
    print(item)


Vamos conferir um exemplo completo de treinamento de modelos do tf.keras com o dataset ORC baseado no dataset Iris.

### Pré-processamento de dados

Configure quais colunas são características e qual é o rótulo:

In [47]:
feature_cols = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
label_cols = ["species"]

# select feature columns
feature_dataset = tfio.IODataset.from_orc("iris.orc", columns=feature_cols)
# select label columns
label_dataset = tfio.IODataset.from_orc("iris.orc", columns=label_cols)

Uma função utilitária para mapear espécies para números de ponto flutuante para o treinamento do modelo:

In [48]:
vocab_init = tf.lookup.KeyValueTensorInitializer(
    keys=tf.constant(["virginica", "versicolor", "setosa"]),
    values=tf.constant([0, 1, 2], dtype=tf.int64))
vocab_table = tf.lookup.StaticVocabularyTable(
    vocab_init,
    num_oov_buckets=4)

In [49]:
label_dataset = label_dataset.map(vocab_table.lookup)
dataset = tf.data.Dataset.zip((feature_dataset, label_dataset))
dataset = dataset.batch(1)

def pack_features_vector(features, labels):
    """Pack the features into a single array."""
    features = tf.stack(list(features), axis=1)
    return features, labels

dataset = dataset.map(pack_features_vector)

## Crie, compile e treine o modelo

Finalmente, está tudo pronto para criar o modelo e treiná-lo! Você criará um modelo do Keras com 3 camadas para prever a classe da planta íris usando o dataset processado.

In [50]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(
            10, activation=tf.nn.relu, input_shape=(4,)
        ),
        tf.keras.layers.Dense(10, activation=tf.nn.relu),
        tf.keras.layers.Dense(3),
    ]
)

model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
model.fit(dataset, epochs=5)